In [1]:
import scipy
import numpy as np
from itertools import product
from matplotlib import pyplot as plt
from scipy.optimize import LinearConstraint, Bounds

In [2]:
shipping_cost = np.array([[1,3],[3,2],[2,2]]).reshape(3,2)
production_cost = np.array([6,5]).reshape(1,2)
given_demand = np.array([[50,60,75],[75,90,100],[60,75,90]]).reshape(3,3)
produce_limit = np.array([250,250]).reshape(2,1)

In [3]:
Gamma = 1
# Gamma = 2

In [4]:
products = (list(product(range(3),repeat=3)))
senarios = [[given_demand[i][j]  for i,j in enumerate(pro)] for pro in products]

destruct_senarios = np.random.binomial(1,0.3,(1,3,2))
destruct_units = np.where(destruct_senarios == 1, 0.5, 1)

num_senario = len(senarios)
num_dest_senario = len(destruct_senarios)

#create empty numpy array
Resource_lft = np.array([])
demand_array = np.array([])

for demand in senarios:
    for destruct in destruct_units:
        transport_mat = np.zeros((3,6))

        for ind,i in enumerate(demand):
            transport_mat[ind,2*ind] = destruct[ind][0]
            transport_mat[ind,2*ind+1] = destruct[ind][1]
            demand_array =  np.append(demand_array,i)

        Resource_lft = np.append(Resource_lft,transport_mat)
Resource_lft = Resource_lft.reshape(num_dest_senario*num_senario*3,6)


# Shortfall Matrix 
Shortfall_lft = np.eye(num_senario*num_dest_senario*3)

# Allocation Matrix
Resource_lft = Resource_lft.reshape(num_dest_senario*num_senario*3,6)

# Demand Matrix
demand_rgt = demand_array.reshape(num_dest_senario*num_senario*3,1)

# Production Matrix
Production_lft = np.repeat([[0,0],[0,0],[0,0]],num_dest_senario*num_senario,axis=0)

# Constraint Matrix
A1_ub = -1*np.concatenate((Production_lft,Resource_lft,Shortfall_lft),axis=1)
b1_ub = -1*demand_rgt

# Balance Matrix
Balance_lft = np.array([[-1,0,1,0,1,0,1,0],[0,-1,0,1,0,1,0,1]])
Dummy_lft = np.zeros((2,len(Shortfall_lft)))

# Concatenate
A2_ub = np.concatenate((Balance_lft,Dummy_lft),axis=1)
b2_ub = np.array([0,0]).reshape(2,1)

A_combined = np.concatenate((A1_ub,A2_ub),axis=0)
b_combined = np.concatenate((b1_ub,b2_ub),axis=0)

A_ex= np.zeros((len(A_combined),num_senario*num_dest_senario))

A_combined = np.concatenate((A_combined,A_ex),axis=1)


In [5]:
A1_ub.shape

(81, 89)

In [6]:
A2_ub.shape

(2, 89)

In [7]:
A_combined.shape

(83, 116)

In [8]:
A1_ub.shape

(81, 89)

In [9]:
A_combined[-2:]

array([[-1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [10]:
A_combined

array([[-0. , -0. , -0.5, ...,  0. ,  0. ,  0. ],
       [-0. , -0. , -0. , ...,  0. ,  0. ,  0. ],
       [-0. , -0. , -0. , ...,  0. ,  0. ,  0. ],
       ...,
       [-0. , -0. , -0. , ...,  0. ,  0. ,  0. ],
       [-1. ,  0. ,  1. , ...,  0. ,  0. ,  0. ],
       [ 0. , -1. ,  0. , ...,  0. ,  0. ,  0. ]])

In [11]:
new_row =[]
for i in production_cost:
    new_row.extend(i)

for i in shipping_cost:
    for j in i:
        new_row.extend([j])

deploy_move_cost = np.repeat([new_row],num_senario*num_dest_senario,axis=0)

new_unstatisfied_demand_costs = np.kron(40*np.eye(num_dest_senario*num_senario, dtype=int), np.ones((1, 3), dtype=int))

overall_cost = -1*np.eye(num_senario*num_dest_senario)

A3_ub = np.concatenate((deploy_move_cost,new_unstatisfied_demand_costs,overall_cost),axis=1)
b3_ub = np.zeros(len(A3_ub)).reshape(-1,1)

In [12]:
print(A3_ub.shape)

(27, 116)


In [13]:
print(A3_ub)

[[ 6.  5.  1. ... -0. -0. -0.]
 [ 6.  5.  1. ... -0. -0. -0.]
 [ 6.  5.  1. ... -0. -0. -0.]
 ...
 [ 6.  5.  1. ... -1. -0. -0.]
 [ 6.  5.  1. ... -0. -1. -0.]
 [ 6.  5.  1. ... -0. -0. -1.]]


In [14]:
A_ub = np.concatenate((A_combined,A3_ub),axis=0)
b_ub = np.concatenate((b_combined,b3_ub),axis=0)

In [15]:
A3_ub[0]

array([ 6.,  5.,  1.,  3.,  3.,  2.,  2.,  2., 40., 40., 40.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.])

In [16]:
new_unstatisfied_demand_costs.shape

(27, 81)

In [17]:
bounds = []

for i in produce_limit:
    bounds.append([0,i[0]])

for i in shipping_cost:
    for j in i:
        bounds.append([0,250])

for i in range(3*num_senario*num_dest_senario):
    bounds.append([0,float("inf")])

for i in range(num_senario*num_dest_senario):

    bounds.append([0,float("inf")])

In [18]:
def utility_val(x):             # calculates the value and the gradient of the expected utility function
    
    # utility function realizations, will be used for Hessian calculations

    length = len(x)-num_dest_senario*num_senario
    
    Realizations = np.exp(Gamma*x[-num_dest_senario*num_senario:])
    val = sum(Realizations)/num_dest_senario*num_senario
    gradient = (Gamma/num_dest_senario*num_senario) * Realizations
    gradient = np.append(np.zeros(length),gradient,axis=0)
    return (val,gradient)
         
def utility_Hp(x,p):            # returns Hessian of the expected utility function multiplied by a vector p
                                # this is easy because the Hessian is diagonal
    Realizations = np.exp(Gamma*x[-num_dest_senario*num_senario:])
    Hp =  Realizations * p[-num_dest_senario*num_senario:] * (Gamma**2/num_dest_senario*num_senario)
    length = len(x)-num_dest_senario*num_senario
    Hp = np.append(np.zeros(length),Hp,axis=0)
    return Hp

In [19]:
l_ub = np.full((len(b_ub),),0.0)
l_ub[-num_dest_senario*num_senario-2:-num_dest_senario*num_senario] = -np.inf

In [20]:
len(l_ub[(-num_dest_senario*num_senario) -2:-num_dest_senario*num_senario])

2

In [21]:
constraints = LinearConstraint(A_ub, l_ub.reshape(-1), b_ub.reshape(-1))
bounds = Bounds(np.array(bounds)[:,0], np.array(bounds)[:,1])

In [22]:
z0 = np.zeros((len(A_ub[0])))

In [23]:
len(z0)

116

In [24]:
SolutionVec = scipy.optimize.minimize(utility_val, z0, method='trust-constr', jac=True, hessp=utility_Hp,
               constraints = constraints, options={'verbose': 1,'maxiter': 500}, bounds=bounds)

c:\Users\aayus\Documents\GitHub\StochOpt\.venv\lib\site-packages\scipy\optimize\_trustregion_constr\equality_constrained_sqp.py:217: UserWarning: Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.
  Z, LS, Y = projections(A, factorization_method)


The maximum number of function evaluations is exceeded.
Number of iterations: 500, function evaluations: 500, CG iterations: 1011, optimality: 1.23e-02, constraint violation: 6.84e+01, execution time: 1.2e+02 s.


In [25]:
SolutionVec.x

array([-5.39273101, -3.23206765, 10.64292524,  6.32383044, 14.65754286,
       17.965146  , 13.41132126, 14.53247021, -0.67236334, -1.04831498,
       -1.18569166, -0.71122968, -1.11092549, -1.00729005, -1.05022113,
       -1.03400978, -0.79934772, -0.75947293, -0.94914577, -1.18738694,
       -0.96284141, -1.12696252, -0.89765751, -0.8189641 , -1.3523178 ,
       -0.77258581, -1.05131037, -0.83073342, -1.0109963 , -0.99734398,
       -0.77728111, -1.18578777, -1.48736952, -0.75427116, -0.66142751,
       -0.38871785, -1.26570357, -1.257252  , -0.59648784, -1.09584973,
       -1.20484548, -0.78497997, -1.26889182, -0.85789138, -0.42680759,
       -1.17495826, -1.24580644, -0.63512009, -0.8087322 , -1.48896608,
       -0.77514863, -1.05929841, -1.10022761, -1.141246  , -0.82839187,
       -0.93234627, -1.29421083, -0.44462904, -1.13673613, -1.12211306,
       -0.90694116, -0.8437028 , -0.29647517, -1.18683668, -1.39996335,
       -0.38776113, -1.41082582, -1.11072094, -0.69526587, -1.33

In [26]:
b_ub.shape

(110, 1)